In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
df = pd.read_csv("../model_preds/preds.csv")

In [3]:
df.head()

,Gender,Married,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Total_Income,Semiurban,Urban,Loan_Status,total_income_by_loanAmt,Loan_Amount_Term_By_amt,kfold,rf_probs,cat_probs
0,1.0,1.0,0.000000,1.0,0.0,104.000000,360.000000,1.0,3819.000000,0.000000,1.0,1,1348.443140,0.288889,0,0.906667,0.941281
1,1.0,0.0,1.000000,1.0,0.0,117.000000,360.000000,1.0,6177.000000,0.000000,1.0,1,2787.298488,0.325000,0,0.803333,0.877542
2,1.0,1.0,2.419781,1.0,0.0,182.901096,334.813149,1.0,7142.630796,0.419781,0.0,0,1525.048970,0.546278,0,0.163333,0.059495
3,1.0,1.0,0.000000,1.0,0.0,143.701687,228.983805,0.0,7818.452295,0.000000,0.0,0,2960.178157,0.627563,0,0.150000,0.038194
4,1.0,1.0,0.000000,1.0,0.0,100.000000,360.000000,1.0,3539.000000,0.000000,1.0,1,1252.452100,0.277778,0,0.826667,0.935005


In [4]:
targets = df.Loan_Status.values

In [5]:
best_params_rf = {'criterion': 'gini', 'max_depth': None, 'n_estimators': 300}

In [6]:
rf_model = RandomForestClassifier(**best_params_rf, n_jobs=-1)

In [7]:
cat_model = CatBoostClassifier()

In [8]:
models = [('rf_model', rf_model), ('cat_model', cat_model)]

In [9]:
print(f"Random Forest Roc-Auc score: {roc_auc_score(targets, df.rf_probs)}")
print(f"Cat Boost Roc-Auc score: {roc_auc_score(targets, df.cat_probs)}")

Random Forest Roc-Auc score: 0.9111021083230312
Cat Boost Roc-Auc score: 0.8867820508536871


# Ensembling

In [10]:
from sklearn.ensemble import VotingClassifier

In [11]:
clf = VotingClassifier(models, voting='soft', n_jobs=-1,)

In [12]:
def ensemble_train(df, clf, fold):
    train_df = df.loc[df.kfold!=fold]
    valid_df = df.loc[df.kfold==fold]
    
    X_train = train_df.iloc[:, :-5].values
    X_valid = valid_df.iloc[:, :-5].values
    
    y_train = train_df.Loan_Status.values
    y_valid = valid_df.Loan_Status.values
    
    clf.fit(X_train, y_train)
    probs = clf.predict_proba(X_valid)[:,1]
    
    auc = roc_auc_score(y_valid, probs)
    return auc

In [13]:
aucs = []
for i in range(5):
    auc = ensemble_train(df, clf, i)
    aucs.append(auc)

In [14]:
np.mean(aucs)

1.0

## Blending these Models

In [15]:
pred_cols = ["rf_probs", "cat_probs"]

## 1. Averaging

In [16]:
avg_probs = np.mean(df[pred_cols].values, axis=1)

In [17]:
roc_auc_score(targets, avg_probs)

0.9018844598884025

## 2. Weighted Averaging

In [18]:
rf_probs = df.rf_probs
cat_probs = df.cat_probs

In [33]:
w_avg_probs = (2*rf_probs.values + 1*cat_probs.values)/ 3

In [34]:
roc_auc_score(targets, w_avg_probs)

0.904924989140967

## 3. Rank Averaging

In [35]:
r_avg_probs = (rf_probs.rank().values + cat_probs.rank().values)/2

In [36]:
roc_auc_score(targets, r_avg_probs)

0.9003976076714892

## 4. Weighted Rank averaging

In [37]:
wr_avg_probs = (2*rf_probs.rank().values + cat_probs.rank().values)/3

In [38]:
roc_auc_score(targets, wr_avg_probs)

0.9046994553777272

## 5. Optimized Coefficients

In [39]:
from functools import partial
from scipy.optimize import fmin

In [40]:
class OptimizeAUC:
    def __init__(self):
        self.coefs_ = 0
        
    def _auc(self, coefs, X, y):
        x_coef = X*coefs
        predictions = np.sum(x_coef, axis=1)
        auc_score = roc_auc_score(y, predictions)
        return -1.0*auc_score
        
    def fit(self, X, y):
        partial_loss = partial(self._auc, X=X, y=y)
        init_coefs = np.random.dirichlet(np.ones(X.shape[1]))
        self.coefs_ = fmin(partial_loss, init_coefs, disp=True)
        
    def predict(self, X):
        x_coef = X * self.coefs_
        predictions = np.sum(x_coef, axis=1)
        return predictions

In [41]:
def run_training_w_avg(df, pred_cols, fold):
    train_df = df.loc[df.kfold!=fold]
    valid_df = df.loc[df.kfold==fold]
    
    X_train = train_df[pred_cols].values
    X_valid = valid_df[pred_cols].values
    
    y_train = train_df.Loan_Status.values
    y_valid = valid_df.Loan_Status.values
    
    opt = OptimizeAUC()
    opt.fit(X_train, y_train)
    
    probs = opt.predict(X_valid)
    auc = roc_auc_score(y_valid, probs)
    print(f"fold= {fold} Auc = {auc}")

    return opt.coefs_

In [42]:
coefs = []
for i in range(5):
    coef = run_training_w_avg(df, pred_cols, i)
    coefs.append(coef)

Optimization terminated successfully.
         Current function value: -0.919871
         Iterations: 28
         Function evaluations: 69
fold= 0 Auc = 0.8950310559006212
Optimization terminated successfully.
         Current function value: -0.921258
         Iterations: 33
         Function evaluations: 92
fold= 1 Auc = 0.8942028985507248
Optimization terminated successfully.
         Current function value: -0.906411
         Iterations: 28
         Function evaluations: 63
fold= 2 Auc = 0.9437093047679059
Optimization terminated successfully.
         Current function value: -0.919991
         Iterations: 33
         Function evaluations: 79
fold= 3 Auc = 0.8935097668557026
Optimization terminated successfully.
         Current function value: -0.907688
         Iterations: 31
         Function evaluations: 67
fold= 4 Auc = 0.9462297836588951


In [43]:
coefs

[array([ 1.34193182, -0.27900847]),
 array([ 8.85133758, -1.71617231]),
 array([ 0.89332485, -0.23445423]),
 array([ 1.82279315, -0.384031  ]),
 array([ 0.32251306, -0.09132349])]

In [44]:
coefs = np.mean(coefs, axis=0)
print(coefs)

[ 2.64638009 -0.5409979 ]


In [45]:
ow_avg_probs = (coefs[0]*rf_probs.values + coefs[1]*cat_probs.values)

In [46]:
roc_auc_score(targets, ow_avg_probs)

0.9147482374954059

## 6. Optimizing rank averaging

In [47]:
def run_training_rank_avg(df, pred_cols, fold):
    train_df = df.loc[df.kfold!=fold]
    valid_df = df.loc[df.kfold==fold]
    
    X_train = train_df[pred_cols].rank().values
    X_valid = valid_df[pred_cols].rank().values
    
    y_train = train_df.Loan_Status.values
    y_valid = valid_df.Loan_Status.values
    
    opt = OptimizeAUC()
    opt.fit(X_train, y_train)
    
    probs = opt.predict(X_valid)
    auc = roc_auc_score(y_valid, probs)
    print(f"fold= {fold} Auc = {auc}")

    return opt.coefs_

In [48]:
coefs = []
for i in range(5):
    coef = run_training_rank_avg(df, pred_cols, i)
    coefs.append(coef)

Optimization terminated successfully.
         Current function value: -0.921219
         Iterations: 28
         Function evaluations: 65
fold= 0 Auc = 0.8954451345755694
Optimization terminated successfully.
         Current function value: -0.919073
         Iterations: 35
         Function evaluations: 92
fold= 1 Auc = 0.8888198757763975
Optimization terminated successfully.
         Current function value: -0.905746
         Iterations: 30
         Function evaluations: 59
fold= 2 Auc = 0.9455996639361478
Optimization terminated successfully.
         Current function value: -0.920030
         Iterations: 28
         Function evaluations: 60
fold= 3 Auc = 0.889939088426801
Optimization terminated successfully.
         Current function value: -0.903231
         Iterations: 19
         Function evaluations: 59
fold= 4 Auc = 0.9466498634740601


In [49]:
coefs = np.mean(coefs, axis=0)
print(coefs)

[ 5.14950938 -0.34403153]


In [50]:
rw_avg_probs = (coefs[0]*rf_probs.values + coefs[1]*cat_probs.values)

In [51]:
roc_auc_score(targets, rw_avg_probs)

0.9122590129974272

## 7. Stacking

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [57]:
def run_training_stack(df, model, pred_cols, fold):
    train_df = df.loc[df.kfold!=fold]
    valid_df = df.loc[df.kfold==fold]
    
    X_train = train_df[pred_cols].values
    X_valid = valid_df[pred_cols].values
    
    y_train = train_df.Loan_Status.values
    y_valid = valid_df.Loan_Status.values
    
    model.fit(X_train, y_train)
    
    probs = model.predict_proba(X_valid)[:,1]
    auc = roc_auc_score(y_valid, probs)
    print(f"fold= {fold} Auc = {auc}")

    return auc

In [58]:
stackers = [LogisticRegression(), RandomForestClassifier(n_jobs=-1),
           GradientBoostingClassifier(), AdaBoostClassifier(), XGBClassifier(),
           CatBoostClassifier()]

In [59]:
names = ["Logistic Regression", "Random Forest", "Gradient Boost", "AdaBoost", "XGB", "catboost"]

In [60]:
model_scores = []
for name, stacker in zip(names, stackers):
    aucs=[]
    for i in range(5):
        auc = run_training_stack(df, stacker, pred_cols, i)
        aucs.append(auc)
    mean_score = np.mean(aucs)
    print(f"{name} Auc score: {mean_score}")
    model_scores.append(mean_score)

fold= 0 Auc = 0.8799171842650103
fold= 1 Auc = 0.8763975155279503
fold= 2 Auc = 0.9428691451375762
fold= 3 Auc = 0.8928796471329552
fold= 4 Auc = 0.941818945599664
Logistic Regression Auc score: 0.9067764875326312
fold= 0 Auc = 0.882815734989648
fold= 1 Auc = 0.8544513457556935
fold= 2 Auc = 0.9449695442134006
fold= 3 Auc = 0.8676748582230625
fold= 4 Auc = 0.9621928166351607
Random Forest Auc score: 0.9024208599633929
fold= 0 Auc = 0.8706004140786751
fold= 1 Auc = 0.8860248447204969
fold= 2 Auc = 0.9366729678638942
fold= 3 Auc = 0.8711405166981726
fold= 4 Auc = 0.966183574879227
Gradient Boost Auc score: 0.9061244636480932
fold= 0 Auc = 0.8865424430641822
fold= 1 Auc = 0.8967908902691512
fold= 2 Auc = 0.9400336063852133
fold= 3 Auc = 0.8770216341104811
fold= 4 Auc = 0.9656584751102709
AdaBoost Auc score: 0.9132094097878596
[16:47:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used w

148:	learn: 0.3548339	total: 801ms	remaining: 4.57s
149:	learn: 0.3543390	total: 807ms	remaining: 4.57s
150:	learn: 0.3539009	total: 812ms	remaining: 4.56s
151:	learn: 0.3533361	total: 817ms	remaining: 4.56s
152:	learn: 0.3526558	total: 822ms	remaining: 4.55s
153:	learn: 0.3521821	total: 826ms	remaining: 4.54s
154:	learn: 0.3517432	total: 832ms	remaining: 4.53s
155:	learn: 0.3512022	total: 837ms	remaining: 4.53s
156:	learn: 0.3508098	total: 841ms	remaining: 4.52s
157:	learn: 0.3503344	total: 846ms	remaining: 4.51s
158:	learn: 0.3496564	total: 851ms	remaining: 4.5s
159:	learn: 0.3491689	total: 856ms	remaining: 4.49s
160:	learn: 0.3488165	total: 860ms	remaining: 4.48s
161:	learn: 0.3482684	total: 865ms	remaining: 4.47s
162:	learn: 0.3478298	total: 870ms	remaining: 4.46s
163:	learn: 0.3473968	total: 875ms	remaining: 4.46s
164:	learn: 0.3470322	total: 879ms	remaining: 4.45s
165:	learn: 0.3466198	total: 883ms	remaining: 4.44s
166:	learn: 0.3462667	total: 888ms	remaining: 4.43s
167:	learn: 0

317:	learn: 0.3073310	total: 1.63s	remaining: 3.5s
318:	learn: 0.3071343	total: 1.64s	remaining: 3.49s
319:	learn: 0.3069505	total: 1.64s	remaining: 3.49s
320:	learn: 0.3067161	total: 1.65s	remaining: 3.49s
321:	learn: 0.3065328	total: 1.65s	remaining: 3.48s
322:	learn: 0.3063661	total: 1.66s	remaining: 3.48s
323:	learn: 0.3062437	total: 1.67s	remaining: 3.48s
324:	learn: 0.3060699	total: 1.67s	remaining: 3.48s
325:	learn: 0.3058788	total: 1.68s	remaining: 3.47s
326:	learn: 0.3056988	total: 1.69s	remaining: 3.47s
327:	learn: 0.3056006	total: 1.69s	remaining: 3.47s
328:	learn: 0.3054807	total: 1.7s	remaining: 3.46s
329:	learn: 0.3052717	total: 1.71s	remaining: 3.46s
330:	learn: 0.3050924	total: 1.71s	remaining: 3.46s
331:	learn: 0.3049509	total: 1.72s	remaining: 3.45s
332:	learn: 0.3047859	total: 1.72s	remaining: 3.45s
333:	learn: 0.3046247	total: 1.73s	remaining: 3.44s
334:	learn: 0.3043385	total: 1.73s	remaining: 3.44s
335:	learn: 0.3041514	total: 1.74s	remaining: 3.43s
336:	learn: 0.

482:	learn: 0.2839998	total: 2.49s	remaining: 2.66s
483:	learn: 0.2839281	total: 2.49s	remaining: 2.65s
484:	learn: 0.2837878	total: 2.5s	remaining: 2.65s
485:	learn: 0.2837303	total: 2.5s	remaining: 2.65s
486:	learn: 0.2836708	total: 2.51s	remaining: 2.64s
487:	learn: 0.2835469	total: 2.51s	remaining: 2.64s
488:	learn: 0.2833529	total: 2.52s	remaining: 2.63s
489:	learn: 0.2831799	total: 2.52s	remaining: 2.63s
490:	learn: 0.2831046	total: 2.53s	remaining: 2.62s
491:	learn: 0.2829629	total: 2.53s	remaining: 2.62s
492:	learn: 0.2829402	total: 2.54s	remaining: 2.61s
493:	learn: 0.2828090	total: 2.55s	remaining: 2.61s
494:	learn: 0.2826875	total: 2.55s	remaining: 2.6s
495:	learn: 0.2825732	total: 2.56s	remaining: 2.6s
496:	learn: 0.2825098	total: 2.56s	remaining: 2.59s
497:	learn: 0.2824035	total: 2.57s	remaining: 2.59s
498:	learn: 0.2822839	total: 2.57s	remaining: 2.58s
499:	learn: 0.2821278	total: 2.58s	remaining: 2.58s
500:	learn: 0.2819579	total: 2.58s	remaining: 2.57s
501:	learn: 0.28

659:	learn: 0.2640740	total: 3.48s	remaining: 1.79s
660:	learn: 0.2639516	total: 3.49s	remaining: 1.79s
661:	learn: 0.2639052	total: 3.5s	remaining: 1.78s
662:	learn: 0.2638602	total: 3.5s	remaining: 1.78s
663:	learn: 0.2638184	total: 3.51s	remaining: 1.77s
664:	learn: 0.2637171	total: 3.51s	remaining: 1.77s
665:	learn: 0.2636794	total: 3.52s	remaining: 1.76s
666:	learn: 0.2636026	total: 3.52s	remaining: 1.76s
667:	learn: 0.2634232	total: 3.53s	remaining: 1.75s
668:	learn: 0.2633461	total: 3.54s	remaining: 1.75s
669:	learn: 0.2632262	total: 3.54s	remaining: 1.74s
670:	learn: 0.2631570	total: 3.55s	remaining: 1.74s
671:	learn: 0.2630458	total: 3.55s	remaining: 1.73s
672:	learn: 0.2629903	total: 3.56s	remaining: 1.73s
673:	learn: 0.2628542	total: 3.56s	remaining: 1.72s
674:	learn: 0.2626158	total: 3.57s	remaining: 1.72s
675:	learn: 0.2625417	total: 3.58s	remaining: 1.71s
676:	learn: 0.2624232	total: 3.58s	remaining: 1.71s
677:	learn: 0.2623552	total: 3.59s	remaining: 1.7s
678:	learn: 0.2

...

117:	learn: 0.4021450	total: 627ms	remaining: 4.69s
118:	learn: 0.4015897	total: 632ms	remaining: 4.67s
119:	learn: 0.4007979	total: 636ms	remaining: 4.66s
120:	learn: 0.3998964	total: 639ms	remaining: 4.64s
121:	learn: 0.3991510	total: 643ms	remaining: 4.63s
122:	learn: 0.3987241	total: 646ms	remaining: 4.61s
123:	learn: 0.3982340	total: 649ms	remaining: 4.59s
124:	learn: 0.3975328	total: 653ms	remaining: 4.57s
125:	learn: 0.3969507	total: 657ms	remaining: 4.55s
126:	learn: 0.3965343	total: 660ms	remaining: 4.54s
127:	learn: 0.3959475	total: 664ms	remaining: 4.52s
128:	learn: 0.3951627	total: 668ms	remaining: 4.51s
129:	learn: 0.3945702	total: 672ms	remaining: 4.5s
130:	learn: 0.3940189	total: 677ms	remaining: 4.49s
131:	learn: 0.3935281	total: 680ms	remaining: 4.47s
132:	learn: 0.3929402	total: 685ms	remaining: 4.46s
133:	learn: 0.3922331	total: 693ms	remaining: 4.48s
134:	learn: 0.3916595	total: 701ms	remaining: 4.49s
135:	learn: 0.3910628	total: 709ms	remaining: 4.51s
136:	learn: 0

319:	learn: 0.3407875	total: 1.45s	remaining: 3.07s
320:	learn: 0.3406337	total: 1.45s	remaining: 3.07s
321:	learn: 0.3405748	total: 1.45s	remaining: 3.06s
322:	learn: 0.3404347	total: 1.46s	remaining: 3.05s
323:	learn: 0.3402758	total: 1.46s	remaining: 3.05s
324:	learn: 0.3399601	total: 1.46s	remaining: 3.04s
325:	learn: 0.3397847	total: 1.47s	remaining: 3.04s
326:	learn: 0.3396600	total: 1.47s	remaining: 3.03s
327:	learn: 0.3394888	total: 1.48s	remaining: 3.02s
328:	learn: 0.3393961	total: 1.48s	remaining: 3.02s
329:	learn: 0.3392347	total: 1.48s	remaining: 3.01s
330:	learn: 0.3391381	total: 1.49s	remaining: 3s
331:	learn: 0.3390226	total: 1.49s	remaining: 3s
332:	learn: 0.3388854	total: 1.49s	remaining: 2.99s
333:	learn: 0.3387755	total: 1.5s	remaining: 2.99s
334:	learn: 0.3386691	total: 1.5s	remaining: 2.98s
335:	learn: 0.3385285	total: 1.5s	remaining: 2.98s
336:	learn: 0.3383758	total: 1.51s	remaining: 2.97s
337:	learn: 0.3382368	total: 1.51s	remaining: 2.96s
338:	learn: 0.3381257

509:	learn: 0.3179581	total: 2.26s	remaining: 2.17s
510:	learn: 0.3179147	total: 2.27s	remaining: 2.17s
511:	learn: 0.3176902	total: 2.27s	remaining: 2.16s
512:	learn: 0.3175786	total: 2.27s	remaining: 2.16s
513:	learn: 0.3175073	total: 2.28s	remaining: 2.15s
514:	learn: 0.3174636	total: 2.28s	remaining: 2.15s
515:	learn: 0.3173132	total: 2.28s	remaining: 2.14s
516:	learn: 0.3171769	total: 2.29s	remaining: 2.14s
517:	learn: 0.3170267	total: 2.29s	remaining: 2.13s
518:	learn: 0.3169520	total: 2.29s	remaining: 2.13s
519:	learn: 0.3168706	total: 2.3s	remaining: 2.12s
520:	learn: 0.3167899	total: 2.3s	remaining: 2.12s
521:	learn: 0.3165763	total: 2.31s	remaining: 2.11s
522:	learn: 0.3165088	total: 2.31s	remaining: 2.11s
523:	learn: 0.3164202	total: 2.31s	remaining: 2.1s
524:	learn: 0.3163519	total: 2.32s	remaining: 2.1s
525:	learn: 0.3162296	total: 2.32s	remaining: 2.09s
526:	learn: 0.3160727	total: 2.33s	remaining: 2.09s
527:	learn: 0.3159894	total: 2.33s	remaining: 2.08s
528:	learn: 0.31

686:	learn: 0.2999176	total: 2.94s	remaining: 1.34s
687:	learn: 0.2998137	total: 2.95s	remaining: 1.34s
688:	learn: 0.2996942	total: 2.95s	remaining: 1.33s
689:	learn: 0.2995913	total: 2.95s	remaining: 1.33s
690:	learn: 0.2994918	total: 2.96s	remaining: 1.32s
691:	learn: 0.2994293	total: 2.96s	remaining: 1.32s
692:	learn: 0.2993139	total: 2.96s	remaining: 1.31s
693:	learn: 0.2992041	total: 2.97s	remaining: 1.31s
694:	learn: 0.2990118	total: 2.97s	remaining: 1.3s
695:	learn: 0.2988479	total: 2.98s	remaining: 1.3s
696:	learn: 0.2987674	total: 2.98s	remaining: 1.3s
697:	learn: 0.2986696	total: 2.98s	remaining: 1.29s
698:	learn: 0.2985871	total: 2.99s	remaining: 1.29s
699:	learn: 0.2984840	total: 2.99s	remaining: 1.28s
700:	learn: 0.2984020	total: 3s	remaining: 1.28s
701:	learn: 0.2983045	total: 3s	remaining: 1.27s
702:	learn: 0.2982596	total: 3s	remaining: 1.27s
703:	learn: 0.2981424	total: 3.01s	remaining: 1.26s
704:	learn: 0.2979600	total: 3.01s	remaining: 1.26s
705:	learn: 0.2978694	to

859:	learn: 0.2810047	total: 3.94s	remaining: 642ms
860:	learn: 0.2808330	total: 3.95s	remaining: 638ms
861:	learn: 0.2807801	total: 3.96s	remaining: 633ms
862:	learn: 0.2807439	total: 3.96s	remaining: 629ms
863:	learn: 0.2806488	total: 3.97s	remaining: 625ms
864:	learn: 0.2805611	total: 3.98s	remaining: 620ms
865:	learn: 0.2804314	total: 3.98s	remaining: 616ms
866:	learn: 0.2803737	total: 3.99s	remaining: 612ms
867:	learn: 0.2802460	total: 4s	remaining: 608ms
868:	learn: 0.2801762	total: 4s	remaining: 603ms
869:	learn: 0.2800012	total: 4.01s	remaining: 599ms
870:	learn: 0.2798648	total: 4.02s	remaining: 595ms
871:	learn: 0.2798121	total: 4.03s	remaining: 591ms
872:	learn: 0.2796415	total: 4.03s	remaining: 587ms
873:	learn: 0.2795539	total: 4.04s	remaining: 582ms
874:	learn: 0.2794522	total: 4.05s	remaining: 578ms
875:	learn: 0.2793574	total: 4.05s	remaining: 574ms
876:	learn: 0.2792914	total: 4.06s	remaining: 569ms
877:	learn: 0.2790880	total: 4.07s	remaining: 565ms
878:	learn: 0.2790

In [62]:
list(zip(names, model_scores))

[('Logistic Regression', 0.9067764875326312),
 ('Random Forest', 0.9024208599633929),
 ('Gradient Boost', 0.9061244636480932),
 ('AdaBoost', 0.9132094097878596),
 ('XGB', 0.8995895220091817),
 ('catboost', 0.9189924085576259)]

#### CatBoost is the best stacker